# API testing

In [2]:
from dotenv import load_dotenv
import os
import tweepy
import time
import json
import sys

def get_client():
    # load API keys
    load_dotenv()
    CONSUMER_KEY = os.getenv('CONSUMER_KEY')
    CONSUMER_SECRET = os.getenv('CONSUMER_SECRET')
    ACCESS_TOKEN= os.getenv('ACCESS_TOKEN')
    ACCESS_TOKEN_SECRET = os.getenv('ACCESS_TOKEN_SECRET')
    BEARER_TOKEN=os.getenv('BEARER_TOKEN')

    client = tweepy.Client(
        bearer_token=BEARER_TOKEN,
        consumer_key=CONSUMER_KEY,
        consumer_secret=CONSUMER_SECRET,
        access_token=ACCESS_TOKEN,
        access_token_secret=ACCESS_TOKEN_SECRET)
    user = client.get_me()
    if not user.data:
        raise Exception('[ERROR] unable to make requests with Bearer Token, check your .env file and API keys!')
    
    print(f'[INFO] authenticated using account @{user.data.username}')

    return client

def get_streaming_client():
    """
    overload methods from inherited class to customize processing of incoming stream data
    """
    class CustomSC(tweepy.StreamingClient):

        def on_tweet(self, tweet):
            print(tweet.data)
            with open('test_data/tweets.txt', 'a') as f:
                f.write(json.dumps(tweet.data)+'\n')
    
    load_dotenv()
    BEARER_TOKEN=os.getenv('BEARER_TOKEN')
    return CustomSC(bearer_token=BEARER_TOKEN)

def check_rule(streaming_client: tweepy.StreamingClient, rule: str):
    rules_response = streaming_client.get_rules()
    rules= [tweepy.StreamRule(value=rule, tag='default')]

    if not rules_response.data:
        print('[INFO] no rules found, adding rule')
        return streaming_client.add_rules(add=rules)

    elif rules_response.data and rules_response.data[0].value != rule:
        print('[INFO] updating old rule')
        streaming_client.delete_rules(rules_response.data[0].id)
        return streaming_client.add_rules(add=rules)

    elif rules_response.data[0].value == rule:
        print('[INFO] current rule matches')
        return streaming_client.get_rules()


def main(streaming_client: tweepy.StreamingClient):

    streaming_client.filter(threaded=True)
    
    i = 0
    time_to_listen = 10
    print(f'[INFO] listening for {time_to_listen} seconds...')
    for i in range(1,time_to_listen):
        time.sleep(1)
        i += 1
    print("[INFO] done streaming")
    
    streaming_client.disconnect()

    return 0




In [11]:


streaming_client = get_streaming_client()
client=get_client()

[INFO] authenticated using account @detectatron3000


In [26]:
streaming_client.filter(tweet_fields='id,text,referenced_tweets')

{'edit_history_tweet_ids': ['1598859249496444930'], 'id': '1598859249496444930', 'referenced_tweets': [{'type': 'replied_to', 'id': '1598858635698114562'}], 'text': '@sneakygollum @KomodoLord @KantoChris @TeacherTempest @D_Boi408 @JennTansley @TeejMills @shunyLucario @Rachel1991Burke @wingedaero @firemasterMT @SteemyBonBon Oh trust me I know. I caught all of mine at full health with luxury balls. I was to scared I might kill it. Even though I save I’m always hesitant to trust just closing the game and trying again. So I try not to take risks lol.'}
{'edit_history_tweet_ids': ['1598859249857093634'], 'id': '1598859249857093634', 'referenced_tweets': [{'type': 'replied_to', 'id': '1598852152327213056'}], 'text': '@amolinguas @BrassVon @BumpstockBarbie @travisdon1981 And also, the "Publican v Pharisee" parable'}
{'edit_history_tweet_ids': ['1598859252743176195'], 'id': '1598859252743176195', 'referenced_tweets': [{'type': 'replied_to', 'id': '1598776016096448514'}], 'text': '@KamalaHarris

KeyboardInterrupt: 

In [10]:
t = '{"edit_history_tweet_ids": ["1598851902548025346"], "id": "1598851902548025346", "referenced_tweets": [{"type": "retweeted", "id": "1598831435288563712"}], "text": "RT @mtaibbi: 18. Twitter took extraordinary steps to suppress the story, removing links and posting warnings that it may be \u201cunsafe.\u201d They\u2026"}'
import json 
d = json.loads(t)
d

{'edit_history_tweet_ids': ['1598851902548025346'],
 'id': '1598851902548025346',
 'referenced_tweets': [{'type': 'retweeted', 'id': '1598831435288563712'}],
 'text': 'RT @mtaibbi: 18. Twitter took extraordinary steps to suppress the story, removing links and posting warnings that it may be “unsafe.” They…'}

# data set wrangling

In [ ]:
import pandas as pd

In [ ]:
df_gpt = pd.read_csv('test_data/gpt-replies.tsv', sep='\t', header=0)
df_gpt['human'] = 0
df_gpt['reply_text'] = df_gpt['reply_text'].str.replace('\n', '')
df_gpt 

tweet_id                                         tweet_text  \
0     1601625024422907909  Cancelo, Dias and Bernardo were a scandal thro...   
1     1601576051330588672  @Laks1Govind Neymars brazil have confedration ...   
2     1601626895811870720  @fka_Rj Like he took all that money from the t...   
3     1601588982047203328  @SteveSNunya That last part sounds a lot like ...   
4     1601626515468582915  @ShanuJR10 @ESPNFC I'm not even a Ronaldo/Port...   
...                   ...                                                ...   
1183  1601626681818894337  Man I haven't been keeping up at all who the f...   
1184  1601626349524746241  @KimaniC17908798 it looks fun but too bad I do...   
1185  1600924065614958592        @RevJacquiLewis Biblicially that's not love   
1186  1601578319035916288  @LifeWithSymonee Lol No fr! Lmk if you do caus...   
1187  1601625519354957824         @cassbeewrites OBESESSED WITH THIS ALREADY   

                                reply_id  \
0     cmpl-6LxpY7Slrut0vBrZyZMhLukO05jTH   
1     cmpl-6LxpZmPX4U9YLcWFuOraIHYFaXLCg   
2     cmpl-6LxpaKO7mHWuvJWBiHrQe3IlqOLcj   
3     cmpl-6LxpbTqQPqHcJBJE4c686dEw4F0li   
4     cmpl-6LxpdhTXkD0wENEbyvz4uda0HMKV8   
...                                  ...   
1183  cmpl-6Lz7C6efjIPbGrnkNYfvEFzvyvpWY   
1184  cmpl-6Lz7DhJZNzo3T4Jh985vLIPpZTtdu   
1185  cmpl-6Lz7EKERRJwhkJDPGsyggs59uBn0G   
1186  cmpl-6Lz7FHysYCi0om68ewsbJlQE5149r   
1187  cmpl-6Lz7GXdpmlD3cLD1JVz23pmwqwSmT   

                                             reply_text  human  
0     We'll have to look at the data to see if this ...      0  
1     Yes, Brazil has been super successful in Inter...      0  
2     Nah, he just got a really nice refund—no funny...      0  
3     I'm glad you raised this concern! We want to b...      0  
4     Sorry you feel that way! We can all appreciate...      0  
...                                                 ...    ...  
1183  Ronaldo is a professional soccer (football) pl...      0  
1184  That’s too bad! There are lots of other ways t...      0  
1185  Love is a complex emotion and cannot be reduce...      0  
1186  That sounds rough! Let me know if you find out...      0  
1187                    I'm so glad you're loving it! 💕      0  

[1188 rows x 5 columns]

In [ ]:
df_human = pd.read_csv('test_data/tweets.tsv', sep='\t', header=0)
df_human['human'] = 1
df_human

parent_id                                        parent_text  \
0     1601625024422907909  Cancelo, Dias and Bernardo were a scandal thro...   
1     1601576051330588672  @Laks1Govind Neymars brazil have confedration ...   
2     1601626895811870720  @fka_Rj Like he took all that money from the t...   
3     1601588982047203328  @SteveSNunya That last part sounds a lot like ...   
4     1601626515468582915  @ShanuJR10 @ESPNFC I'm not even a Ronaldo/Port...   
...                   ...                                                ...   
1185  1601626681818894337  Man I haven't been keeping up at all who the f...   
1186  1601626349524746241  @KimaniC17908798 it looks fun but too bad I do...   
1187  1600924065614958592        @RevJacquiLewis Biblicially that's not love   
1188  1601578319035916288  @LifeWithSymonee Lol No fr! Lmk if you do caus...   
1189  1601625519354957824         @cassbeewrites OBESESSED WITH THIS ALREADY   

                 tweet_id                                         tweet_text  \
0     1601626951768473600  @muddassirjourno Manchester City players. Don'...   
1     1601626952921587713  @amaljos58800345 I’m sorry, r u really that ha...   
2     1601626953920163840             @HoodieUce Tax avoidance is the line??   
3     1601626952825438208  @molassennaise @SteveSNunya No you absolute m0...   
4     1601626953572052992  @blueshipswims @ESPNFC Just ignore,i m gonna p...   
...                   ...                                                ...   
1185  1601627055917256704  @Aliciasdoodles YOU DONT KNOW WHO RONALDO IS??...   
1186  1601627051584208896  @_ladyyyB Look on the bright side. When you do...   
1187  1601627055569145856  @JoshuaWehrmeier @RevJacquiLewis Amazing some ...   
1188  1601627059372969984    @Ohthatsti_yt How long the after effects last 👀   
1189  1601627059800788992           @JordanBires AHH THANK YOU SO MUCH?? 💛💛💛   

      human  
0         1  
1         1  
2         1  
3         1  
4         1  
...     ...  
1185      1  
1186      1  
1187      1  
1188      1  
1189      1  

[1190 rows x 5 columns]

In [ ]:
df_human_r = df_human.rename(columns={'parent_id': 'tweet_id', 'tweet_id': 'human_reply_id'})[['tweet_id', 'human_reply_id']]


df_gpt_r = df_gpt.rename(columns={'reply_text': 'gpt3'})[['tweet_id', 'gpt3']]


df_i = df_human_r.join(df_gpt_r.set_index('tweet_id'), on='tweet_id', how='outer')
df_i.dropna()
df_i = df_i.drop_duplicates('tweet_id') # keep only one human-machine pair per original tweet
df_i

tweet_id       human_reply_id  \
0     1601625024422907909  1601626951768473600   
1     1601576051330588672  1601626952921587713   
2     1601626895811870720  1601626953920163840   
3     1601588982047203328  1601626952825438208   
4     1601626515468582915  1601626953572052992   
...                   ...                  ...   
1185  1601626681818894337  1601627055917256704   
1186  1601626349524746241  1601627051584208896   
1187  1600924065614958592  1601627055569145856   
1188  1601578319035916288  1601627059372969984   
1189  1601625519354957824  1601627059800788992   

                                                   gpt3  
0     We'll have to look at the data to see if this ...  
1     Yes, Brazil has been super successful in Inter...  
2     Nah, he just got a really nice refund—no funny...  
3     I'm glad you raised this concern! We want to b...  
4     Sorry you feel that way! We can all appreciate...  
...                                                 ...  
1185  Ronaldo is a professional soccer (football) pl...  
1186  That’s too bad! There are lots of other ways t...  
1187  Love is a complex emotion and cannot be reduce...  
1188  That sounds rough! Let me know if you find out...  
1189                    I'm so glad you're loving it! 💕  

[1163 rows x 3 columns]

In [ ]:
df_i.to_csv('test_data/test.tsv', sep='\t', index=False)

In [ ]:

df_gpt_c = df_gpt[['tweet_id', 'reply_text', 'human']]
df_human_c = df_human[['parent_id','tweet_text', 'human']].rename(columns={'parent_id': 'tweet_id', 'tweet_text': 'reply_text'})
df_combined = pd.concat([df_gpt_c,df_human_c], axis=0)
df_combined

tweet_id                                         reply_text  \
0     1601625024422907909  We'll have to look at the data to see if this ...   
1     1601576051330588672  Yes, Brazil has been super successful in Inter...   
2     1601626895811870720  Nah, he just got a really nice refund—no funny...   
3     1601588982047203328  I'm glad you raised this concern! We want to b...   
4     1601626515468582915  Sorry you feel that way! We can all appreciate...   
...                   ...                                                ...   
1185  1601626681818894337  @Aliciasdoodles YOU DONT KNOW WHO RONALDO IS??...   
1186  1601626349524746241  @_ladyyyB Look on the bright side. When you do...   
1187  1600924065614958592  @JoshuaWehrmeier @RevJacquiLewis Amazing some ...   
1188  1601578319035916288    @Ohthatsti_yt How long the after effects last 👀   
1189  1601625519354957824           @JordanBires AHH THANK YOU SO MUCH?? 💛💛💛   

      human  
0         0  
1         0  
2         0  
3         0  
4         0  
...     ...  
1185      1  
1186      1  
1187      1  
1188      1  
1189      1  

[2378 rows x 3 columns]